In [8]:
from tkinter import *
from tkinter import ttk
import psycopg2

In [9]:
def commit(method):
    def wrapper(*args, **kw):
        f = method(*args, **kw)
        args[0].connect.commit()
        print('Изменения сохранены')
        return f
    return wrapper

def del_frames(method):
    def wrapper(*args, **kw):
        all_frames = [f for f in args[0].children]
        if all_frames:
            for f_name in all_frames:
                args[0].nametowidget(f_name).destroy()
            print(f'Перед методом {method.__name__} удалились фреймы {all_frames}')
            return method(*args, **kw)
        else:
            print('Нет фреймов, которые можно удалить')
            return method(*args, **kw)
    return wrapper

In [10]:
class Users:
    __instance = None

    def connect_db(self, dbname, user, password, host):
        try:
            self.connect = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host)
        except:
            print('Can`t establish connection to database')
            self.connect = None
        finally:
            return self.connect

    def __new__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)

        return super().__new__(cls)

    def __del__(self):
        Users.__instance = None

    def __init__(self, dbname, user, password, host):
        if self.connect_db(dbname, user, password, host):
            self.dbname = dbname
            self.user = user
            self.password = password
            self.host = host

    def select_users(self):
        """Получаем список всех пользователей"""
        with self.connect.cursor() as curs:
            curs.execute('SELECT * FROM main.users')
            return curs.fetchall()

    def user_exist(self, login):
        """Получаем список всех пользователей"""
        with self.connect.cursor() as curs:
            curs.execute('SELECT * FROM main.users WHERE login = (%s)', (login,))
            return curs.fetchall()

    @commit
    def create_user(self, **kw):
        # Добавляем нового пользователя
        with self.connect.cursor() as curs:
            print(f'Ожидается запись нового пользователя {kw["login"]}')
            return curs.execute('INSERT INTO main.users (first_name, login, password) VALUES (%s, %s, %s)', tuple(kw.values()))

    @commit
    def change_password(self, login=None, password=None):
        """Добавляем нового пользователя"""
        with self.connect.cursor() as curs:
            print(f'Пользователь {login} изменил пароль')
            return curs.execute('UPDATE main.users SET password=%s WHERE login=%s', (password, login))

In [22]:
class App(Tk):

    def __init__(self):
        super().__init__()
        self.title('Auth')
        self.main_page = self.main_page()

    @del_frames
    def main_page(self):
        self.main_page = MainPage().pack()

    @del_frames
    def success_sign_in_page(self, name):
        self.success_sign_in_page = SuccessPage(f'Привет, {name}').pack()

    @del_frames
    def success_sign_up_page(self):
        self.success_sign_up_page = SuccessPage(f'Регистрация прошла успешно').pack()

    @del_frames
    def success_change_password_page(self):
        self.success_change_password_page = SuccessPage(f'Пароль был успешно изменен').pack()

    @del_frames
    def change_password_page(self):
        self.change_password_page = ChangePassword().pack()

    @del_frames
    def sign_up_page(self):
        self.sign_up_page = SignUp().pack()

In [23]:
class MainPage(Frame):

    """Главная страница"""

    def __init__(self):
        super().__init__()
        self.field_login = Field(self, 'Логин')
        self.field_password = Field(self, 'Пароль')
        self.sign_on = Button(self,text = 'Войти', command=self.click_sign_in)
        self.sign_on.pack()
        self.sign_in = Button(self,text = 'Зарегистрироваться', command=self.click_sign_up)
        self.sign_in.pack()
        self.forgot_password = Button(self,text = 'Забыли пароль?', command=self.click_forgot_password)
        self.forgot_password.pack()


    def check_fields(self, e_login, e_password):
        user = db_users.user_exist(e_login)
        if user:
            name = user[0][1]
            if e_password == user[0][-1]:
                self.master.success_sign_in_page(name)
            else:
                self.field_password.info.config(text='Введен неверный пароль')
                self.field_login.info.config(text='')
        else:
            self.field_login.info.config(text='Пользователь не найден')
            self.field_password.info.config(text='')



    def click_sign_in(self):
        e_login = self.field_login.e.get()
        e_password = self.field_password.e.get()
        self.check_fields(e_login, e_password)

    def click_sign_up(self):
        self.master.sign_up_page()

    def click_forgot_password(self):
        self.master.change_password_page()


class SuccessPage(Frame):

    def __init__(self, alert_text):
        super().__init__()
        self.l = Label(self, text = alert_text, font=30)
        self.l.grid(row=0,column=0,columnspan=1)
        self.b = Button(self,text = 'Выйти на главную', command=self.return_back)
        self.b.grid(row=1,column=0,columnspan=1)

    def return_back(self):
        self.master.main_page()


class ChangePassword(Frame):

    """Страница смены пароля"""

    def __init__(self):
        super().__init__()
        self.field_login = Field(self, 'Логин')
        self.field_old_password = Field(self, 'Старый пароль')
        self.field_new_password = Field(self, 'Новый пароль')
        self.field_new_password_repeat = Field(self, 'Повторите новый пароль')
        self.change_password = Button(self,text = 'Зарегистрироваться', command=self.click_change_password)
        self.change_password.pack()
        self.back = Button(self,text = 'Назад', command=self.return_back)
        self.back.pack()

    def get_info(self):
        for var_dict in self.__dir__():
            for k in var_dict:
                print(k)
                var = exec('%s = %d' % (f'self{k}', v))
                print(var)

    def check_fields(self, e_login, e_old_password, e_new_password, e_new_password_repeat):
        user = db_users.user_exist(e_login)
        if user:
            name = user[0][1]
            if e_old_password == user[0][-1]:
                if e_new_password:
                    if len(e_new_password) >= 4:
                        if e_new_password != user[0][-1]:
                            if e_new_password == e_new_password_repeat:
                                db_users.change_password(login=e_login,password=e_new_password)
                                self.master.success_change_password_page()
                            else:
                                self.field_login.info.config(text='')
                                self.field_old_password.info.config(text='')
                                self.field_new_password.info.config(text='')
                                self.field_new_password_repeat.info.config(text='Новый пароль введен неверно')
                        else:
                            self.field_login.info.config(text='')
                            self.field_old_password.info.config(text='')
                            self.field_new_password.info.config(text='Новый пароль не должен совпадать со старым')
                            self.field_new_password_repeat.info.config(text='')
                    else:
                        self.field_login.info.config(text='')
                        self.field_old_password.info.config(text='')
                        self.field_new_password.info.config(text='Новый пароль должен быть длиннее 4 символов')
                        self.field_new_password_repeat.info.config(text='')
                else:
                    self.field_login.info.config(text='')
                    self.field_old_password.info.config(text='')
                    self.field_new_password.info.config(text='Введите пароль')
                    self.field_new_password_repeat.info.config(text='')
            else:
                self.field_login.info.config(text='')
                self.field_old_password.info.config(text='Введен неверный пароль')
                self.field_new_password.info.config(text='')
                self.field_new_password_repeat.info.config(text='')
        else:
            self.get_info()
            self.field_login.info.config(text='Пользователь не найден')
            self.field_old_password.info.config(text='')
            self.field_new_password.info.config(text='')
            self.field_new_password_repeat.info.config(text='')

    def click_change_password(self):
        #Добавить проверку пароля. Цеплять старый пароль из БД
        e_login = self.field_login.e.get()
        e_old_password = self.field_old_password.e.get()
        e_new_password = self.field_new_password.e.get()
        e_new_password_repeat = self.field_new_password_repeat.e.get()
        self.check_fields(e_login, e_old_password, e_new_password, e_new_password_repeat)

    def return_back(self):
        self.master.main_page()

class SignUp(Frame):

    """Страница регистрации"""

    def __init__(self):
        super().__init__()
        self.field_name = Field(self, 'Имя')
        self.field_login = Field(self, 'Логин')
        self.field_password = Field(self, 'Пароль')
        self.sign_up = Button(self,text = 'Зарегистрироваться', command=self.click_sign_up)
        self.sign_up.pack()
        self.back = Button(self,text = 'Назад', command=self.return_back)
        self.back.pack()

    def click_sign_up(self):
        #Добавить проверку логина. Цеплять старый пароль из БД
        self.master.success_sign_up_page()

    def return_back(self):
        self.master.main_page()

In [24]:
class Field(Frame):

    """Фрейм - поле с названием"""

    def __init__(self, master, text):
        super().__init__()
        self.l = Label(master, text = text, font=30)
        self.l.pack()
        self.e = Entry(master)
        self.e.pack()
        self.info = Label(master, text = '')
        self.info.pack()

In [25]:
db_users = Users('users', 'postgres', '1234', 'localhost')
if db_users.connect:
    app = App()
    app.mainloop()

Нет фреймов, которые можно удалить
Перед методом success_sign_in_page удалились фреймы ['!mainpage', '!field', '!field2']


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Nikita\AppData\Local\Temp\ipykernel_16096\735244311.py", line 54, in return_back
    self.master.main_page()
TypeError: 'NoneType' object is not callable
